$\textbf{Empirical Exercise-Week 6}$  Page 402, Problem 8.1
Heteroskedasticity and Labor Market Discrimination 

In [1]:
addpath(genpath('c:\LeSage7'));
Tab1 = readtable('cps5.csv');

In [2]:
Tab1(1,:)


ans =

  1x23 table

    age    asian    black    divorced    educ    exper    faminc    female    hrswork    insure    married    mcaid    mcare    metro    midwest    nchild    northeast    single    south    union    wage     west    white
    ___    _____    _____    ________    ____    _____    ______    ______    _______    ______    _______    _____    _____    _____    _______    ______    _________    ______    _____    _____    _____    ____    _____

    45       0        0         0         13      26      39200       1         38         1          1         0        0        0         0         0           1          0         0        1      14.38     0        1  




In [3]:
wage = table2array(Tab1(:,21)); educ = table2array(Tab1(:,5));
exper = table2array(Tab1(:,6)); female = table2array(Tab1(:,8));
aframer = table2array(Tab1(:,3)); metro = table2array(Tab1(:,14));
south = table2array(Tab1(:,19)); midwest = table2array(Tab1(:,15));
west = table2array(Tab1(:,22));

In [4]:
y = log(wage);
x = [ones(length(y),1) educ exper exper.^2 female aframer metro south midwest west];

In [6]:
results1 = ols(y,x);
[results1.beta results1.tstat]


ans =

    1.2014   37.4089
    0.1012   57.5737
    0.0296   22.7799
   -0.0004  -16.9148
   -0.1655  -17.3680
   -0.1115   -6.5826
    0.1190    9.6711
   -0.0458   -3.3740
   -0.0639   -4.5338
   -0.0066   -0.4575




You see that the coefficient for female = 1 has a depressive effect on wages.  Given equal education, experience, location, women earn less in the United States.
Lets do a Goldfeld-Quandt test on equalty of variances for the residuals of wages for men and women in the US workforce. 

In [12]:
ehat = results1.resid;
index1 = female == 1;
ehat_women = ehat(index1,:);
index2 = female == 0;
ehat_men = ehat(index2,:);


In [11]:
Var_women = var(ehat_women);  Var_men = var(ehat_men);
[Var_women Var_men]


ans =

    0.2113    0.2215




In [13]:
dgf_women = length(ehat_women)-10; dgf_men = length(ehat_men)-10;
F_stat =  (Var_women/dgf_women)/(Var_men/dgf_men)


F_stat =

    1.1829




In [14]:
Fprob = fcdf(F_stat, dgf_women, dgf_men);
1 - Fprob


ans =

   2.3886e-09




We can reject the hypothesis of equaltiy of variances between the men and women wage-equation residuals.

New we do the N $R^2$ test for Heteroskeasticity.  We regress the squared residuals on three variables, on Metro, Female, Black and their cross terms. If the null is true, no heteroskedascity, the term N $R^2$ has a $\chi^{2}$ distribution with K*(K+3)/2 degress of freedom

In [23]:
yy = ehat .^2;
x1 = ones(length(yy),1);

In [16]:
xfa = female .* aframer;
xfm = female .* metro;
xma = metro .* aframer;

In [21]:
xx = [x1 female aframer metro xfa xfm xma];

In [22]:
results2 = ols(yy,xx);

In [24]:
Chistat = results2.rsqr * length(ehat);
dgf2 = 7*10/2;
Chistat



Chistat =

   32.2982




In [53]:
Chip = chi2cdf(Chistat, dgf2);
1 - Chip


ans =

    0.5992




So we caonnot reject heteroskedasticty with these variables in this test.  Let repeat it with all of the explanatory variables, as your book suggests

In [25]:
results3 = ols(yy,x);
dgf4 =  10 * 13/2;
Chistat =  length(yy)*results3.rsqr;
Chip = chi2cdf(Chistat, dgf4);
1-Chip



ans =

   4.7186e-04




We can now reject homoskecasticty when we use all of the regressors.
White test is similar, with quadratic expansion on the variables. 

In [26]:
xxx= [ones(length(yy),1) educ  educ.^2 exper exper.^2 educ.*exper ...
female aframer metro south midwest west female.*aframer metro.*aframer ...
metro.*south metro.*female];
results4 = ols(yy,xxx);
dgf5 = 15;
White = length(yy) * results4.rsqr;

In [27]:
White_test = chi2cdf(White,dgf5);
1 - White_test


ans =

     0




We can reject homoskedasticity with the White test

In [63]:
help hac  % Use the heteroskeastic estimators from the econometrics toolbox;

 HAC Heteroscedasticity and autocorrelation consistent covariance estimators
 
  Syntax:
 
    [EstCoeffCov,se,coeff] = hac(X,y)
    [EstCoeffCov,se,coeff] = hac(Tbl)
    [EstCoeffCov,se,coeff] = hac(Mdl)
    [EstCoeffCov,se,coeff] = hac(...,param,val,...)
 
  Description:
 
    HAC computes robust covariance estimates for ordinary least squares
    (OLS) coefficient estimates of multiple linear regression models
 
    y = X*coeff + e
 
    under general forms of heteroscedasticity and autocorrelation in the
    innovations process e. Estimates are of the form
                
    PhiHat = X'*OmegaHat*X
    EstCoeffCov = c*inv(X'*X)*PhiHat*inv(X'*X)
 
    where OmegaHat is an estimate of the innovations covariance and c is an
    optional small-sample correction.
 
  Input Arguments:  
 
    X - numObs-by-numPreds matrix of predictor data for a multiple linear
        regression model.
 
    y - numObs-by-1 vector of response data for a multiple linear
        regression model.
 
    T

In [30]:
  [EstCoeffCov,se,coeff] = hac(x(:,2:end), y,'type','hc');
  


Estimator type: HC
Estimation method: HC0
Effective sample size: 9799
Small sample correction: off

Coefficient Covariances:

       |  Const      x1       x2       x3       x4       x5       x6       x7       x8       x9   
--------------------------------------------------------------------------------------------------
 Const |  0.0011  -0.0001  -0.0000   0.0000  -0.0000  -0.0000  -0.0001  -0.0001  -0.0001  -0.0001 
 x1    | -0.0001   0.0000  -0.0000   0.0000  -0.0000   0.0000  -0.0000   0.0000   0.0000   0.0000 
 x2    | -0.0000  -0.0000   0.0000  -0.0000   0.0000   0.0000  -0.0000  -0.0000  -0.0000  -0.0000 
 x3    |  0.0000   0.0000  -0.0000   0.0000  -0.0000   0.0000   0.0000   0.0000   0.0000   0.0000 
 x4    | -0.0000  -0.0000   0.0000  -0.0000   0.0001  -0.0000   0.0000  -0.0000  -0.0000   0.0000 
 x5    | -0.0000   0.0000   0.0000   0.0000  -0.0000   0.0003  -0.0000  -0.0000  -0.0000   0.0000 
 x6    | -0.0001  -0.0000  -0.0000   0.0000   0.0000  -0.0000   0.0001  -0.0000  

In [74]:
[coeff se coeff./se] %  coefficients std errors, tstats


ans =

    1.2014    0.0328   36.6527
    0.1012    0.0019   53.1431
    0.0296    0.0013   22.5391
   -0.0004    0.0000  -16.1615
   -0.1655    0.0095  -17.4517
   -0.1115    0.0161   -6.9333
    0.1190    0.0116   10.2814
   -0.0458    0.0139   -3.2931
   -0.0639    0.0137   -4.6615
   -0.0066    0.0145   -0.4529




We see that being female or African American have negative effects on wages.

Now we turn to FGLS estimators, Feasible Generalized Least Squares Estimators


In [75]:
help fgls


 FGLS Feasible Generalized Least Squares
 
  Syntax:
 
    [coeff,se,EstCoeffCov] = fgls(X,y)
    [coeff,se,EstCoeffCov] = fgls(Tbl)
    [coeff,se,EstCoeffCov] = fgls(...,param,val,...)
    [coeff,se,EstCoeffCov,iterPlots] = fgls(...)
    [coeff,se,EstCoeffCov,iterPlots] = fgls(ax,...)
 
  Description:
 
    FGLS computes generalized least squares (GLS) estimates of coefficients
    in multiple linear regression models
 
    y = X*coeff + e
 
    by first estimating the covariance of the innovations process e.
    Coefficient estimates are of the form
                
    coeff = inv(X'*inv(OmegaHat)*X)*X'*inv(OmegaHat)*y
 
    where OmegaHat is an innovations covariance estimate using a specified
    model. The process can be iterated.
 
  Input Arguments:
 
    X - numObs-by-numPreds matrix of predictor data for a multiple linear                                                                                                                                                             

In [28]:
 [coeff1,se1,EstCoeffCov1] = fgls(x(:,2:end),y);

In [29]:
[coeff1 se1 coeff1./se1]


ans =

    1.2235    0.0330   37.1014
    0.0996    0.0018   55.7920
    0.0296    0.0013   22.6168
   -0.0004    0.0000  -16.7470
   -0.1648    0.0093  -17.6437
   -0.1136    0.0174   -6.5376
    0.1193    0.0132    9.0308
   -0.0461    0.0146   -3.1535
   -0.0644    0.0152   -4.2279
   -0.0072    0.0156   -0.4661




We see that the FGLS are similar to the White estimates.  

In [31]:
[coeff coeff1]  % Slight differences


ans =

    1.2014    1.2235
    0.1012    0.0996
    0.0296    0.0296
   -0.0004   -0.0004
   -0.1655   -0.1648
   -0.1115   -0.1136
    0.1190    0.1193
   -0.0458   -0.0461
   -0.0639   -0.0644
   -0.0066   -0.0072




The White estimators are more restrictive than the FGLS estimators.  However it is always good to check for robustness with respect to estimaiton procedure